In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## start with actors > 30 IMDB rating 5yr sum
Just to limit the number of movies to pull from mojo

In [4]:
def get_mojo_film_money(tconst):
    '''
    mojo film website uses IMDB tconst for the url
    url = f'https://www.boxofficemojo.com/title/{tconst}/'
    get film budget by using beautiful soup to parse the summary box
    '''
    try:
        url = f'https://www.boxofficemojo.com/title/{tconst}/'
        response = requests.get(url)
        html = response.text

        soup = BeautifulSoup(html, "html.parser")

        money_list = [span.text.strip() for span in soup.find_all("div", class_="mojo-performance-summary-table")[0].find_all("span", class_="money")]

        label_list = [span.text.strip() for span in soup.find_all("div", class_="mojo-performance-summary-table")[0].find_all("span", class_="a-size-small")]

        for idx, label in enumerate(label_list):
            if 'Domestic' in label:
                df = pd.DataFrame([(tconst, label, money_list[idx])], columns=['tconst','Money_Type','Money_Amount'])
    except:
        df = pd.DataFrame([(tconst, 'Missing', -999)], columns=['tconst','Money_Type','Money_Amount'])
    
    return df


In [8]:
actors_all_year_filtered = pd.read_csv("actors_all_year_30+_with_link.csv")[['nconst']].drop_duplicates()
actors_all_year_filtered.head(3)

,nconst
0,nm3485845
10,nm0001191
31,nm0004778


In [6]:
actor_movies_ratings = pd.read_csv("actor_movies_ratings.csv").drop_duplicates(subset=['tconst','nconst'])
actor_movies_ratings.head(3)

,tconst,ordering,nconst,category,job,characters,averageRating,numVotes,primaryTitle,startYear,region,primaryName,birthYear
0,tt0035423,2,nm0413168,actor,\N,"[""Leopold""]",6.4,88860,Kate & Leopold,2001.0,US,Hugh Jackman,1968
2,tt0120903,2,nm0413168,actor,\N,"[""Logan""]",7.3,644121,X-Men,2000.0,US,Hugh Jackman,1968
8,tt0180037,2,nm0413168,actor,\N,"[""Jack Willis""]",6.4,2344,Paperback Hero,1999.0,US,Hugh Jackman,1968


In [9]:
actor_movies_ratings.shape

(313009, 13)

In [7]:
actor_movies_ratings_filtered = actor_movies_ratings.merge(actors_all_year_filtered, how='inner', on='nconst')
actor_movies_ratings_filtered.head(3)

,tconst,ordering,nconst,category,job,characters,averageRating,numVotes,primaryTitle,startYear,region,primaryName,birthYear
0,tt0035423,2,nm0413168,actor,\N,"[""Leopold""]",6.4,88860,Kate & Leopold,2001.0,US,Hugh Jackman,1968
1,tt0120903,2,nm0413168,actor,\N,"[""Logan""]",7.3,644121,X-Men,2000.0,US,Hugh Jackman,1968
2,tt0180037,2,nm0413168,actor,\N,"[""Jack Willis""]",6.4,2344,Paperback Hero,1999.0,US,Hugh Jackman,1968


In [10]:
actor_movies_ratings_filtered.shape

(6708, 13)

In [11]:
df_list = [get_mojo_film_money(tconst) for tconst in actor_movies_ratings_filtered['tconst'].unique()]
actor_movies_revenue_filtered = pd.concat(df_list)
actor_movies_revenue_filtered.head(3)

,tconst,Money_Type,Money_Amount
0,tt0035423,Domestic (62%),"$47,121,859"
0,tt0120903,Domestic (53.1%),"$157,299,718"
0,tt0180037,Missing,-999


In [12]:
actor_movies_revenue_filtered.to_csv("actor_movies_revenue_filtered.csv", index=False)

In [16]:
(actor_movies_revenue_filtered[actor_movies_revenue_filtered['Money_Type']=='Missing']
 .merge(actor_movies_ratings_filtered[['tconst','primaryTitle']].drop_duplicates()
        , how='inner', on='tconst')
 .to_csv("missing_movie_revenue.csv", index=False))